In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
pd.set_option("display.max_column",100)
pd.set_option("expand_frame_repr",False)

In [3]:
train_data=pd.read_csv(r"D:\文件\学习\数据集\二手车交易\trainDataAfterCleaning.csv",index_col=False)
test_data=pd.read_csv(r"D:\文件\学习\数据集\二手车交易\testDataAfterCleaning.csv",index_col=False)

In [4]:
train_data.head(3)

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14,price
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,0.0,1046,0,0,20160404,43.357796,3.966344,0.050257,2.159744,1.143786,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762,1850
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,0.0,4366,0,0,20160309,45.305273,5.236112,0.137925,1.380657,-1.422165,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522,3600
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,0.0,2806,0,0,20160402,45.978359,4.823792,1.319524,-0.998467,-0.996911,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963,6222


In [5]:
#定义删除字段的函数
def delete(data,col_name):
    data=data.drop(col_name,axis=1)
    return data

In [6]:
#删除训练集合和测试集合的无用字段
train_data=delete(train_data,"SaleID")

In [7]:
train_data=delete(train_data,"name")

In [8]:
data=train_data.drop("price",axis=1)
label=train_data["price"]

In [9]:
#对字段做标准化处理
from sklearn.preprocessing import StandardScaler
data1=StandardScaler().fit_transform(data)
data1=pd.DataFrame(data1)
data1.columns=data.columns

In [10]:
data1.head(3)

,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0.116167,-0.345790,-0.260985,-0.441885,-0.656031,-0.524841,-0.334801,-0.024807,-0.324811,-0.815274,-0.002582,0.0,0.685901,-0.426644,1.101390,-0.010422,1.026863,0.943382,-0.273517,1.102861,0.024107,-1.210454,0.993764,-0.763653,0.850575,-0.963515,0.616959,0.881615
1,-0.072112,-0.143916,-0.896745,0.133083,-0.656031,-0.524841,-0.673462,0.613033,-0.324811,0.945662,-0.002582,0.0,-0.204172,0.365807,1.450046,0.019503,0.642400,-1.206498,0.361833,1.470359,0.054801,-1.080898,-1.160372,-1.298773,0.635193,-0.411241,-1.336973,0.237151
2,0.116186,1.370138,0.883384,-0.441885,-0.656031,-0.524841,0.246567,-0.024807,-0.324811,0.118235,-0.002582,0.0,0.667162,0.639694,1.336830,0.422834,-0.531650,-0.850200,0.069998,1.352636,0.200856,0.138071,-0.978437,-1.284529,0.546095,0.619876,-0.646347,-0.220731


In [11]:
#切分数据集
from sklearn.model_selection import KFold
kf=KFold(n_splits=5)
for train_index,test_index in kf.split(data1):
    x_train=data1.values[train_index]
    x_test=data1.values[test_index]
    y_train=label[train_index]
    y_test=label[test_index]

In [12]:
#建立模型
#决策树
from sklearn.tree import DecisionTreeRegressor
model_dtr=DecisionTreeRegressor()
model_dtr.fit(x_train, y_train)
model_dtr.score(x_test,y_test)

0.9238757456375635

In [13]:
#随机森林
from sklearn.ensemble import RandomForestRegressor
model_rfr=RandomForestRegressor()
model_rfr.fit(x_train, y_train)
model_rfr.score(x_test,y_test)

0.9628807984589852

In [15]:
#支持向量机
from sklearn.svm import SVR
model_svr=SVR(kernel="linear")
model_svr.fit(x_train,y_train)
model_svr.score(x_test, y_test)

0.554888737207246

In [17]:
#xgboost
from xgboost import XGBRegressor
model_xgb=XGBRegressor()
model_xgb.fit(x_train, y_train)
model_xgb.score(x_test, y_test)

0.965672510626447

In [18]:
#KNN
from sklearn.neighbors import KNeighborsRegressor
model_knn=KNeighborsRegressor(n_neighbors=5)
model_knn.fit(x_train,y_train)
model_knn.score(x_test,y_test)

0.9053537695271847

In [33]:
#建立预测函数
from sklearn.metrics import mean_squared_error
def predict_data(model_instance,test):
    y_pred=model_instance.predict(x_test)
    mse=mean_squared_error(y_test, y_pred)
    return print("{}:".format(model_instance),mse)

In [34]:
mse_dtr=predict_data(model_dtr,y_test)

DecisionTreeRegressor(): 4226677.510617021


In [35]:
mse_rfr=predict_data(model_rfr,y_test)

RandomForestRegressor(): 2060984.3167526082


In [37]:
mse_svr=predict_data(model_svr,y_test)

SVR(kernel='linear'): 24714091.19110958


In [38]:
mse_xgb=predict_data(model_xgb,y_test)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None): 1905978.9622417209


In [39]:
mse_knn=predict_data(model_knn,y_test)

KNeighborsRegressor(): 5255080.619896663


In [40]:
#模型融合
#选择随机森林和xgboost进行模型融合

In [41]:
#算数平均模型融合
def Mean_method(test_pred1,test_pred2):
    Mean_result=pd.concat([pd.Series(test_pred1),pd.Series(test_pred2)],axis=1).mean(axis=1)
    return Mean_result

In [43]:
test_pred1=model_rfr.predict(x_test)
test_pred2=model_xgb.predict(x_test)

In [44]:
Mean_pred=Mean_method(test_pred1,test_pred2)

In [45]:
mse_Mean=mean_squared_error(y_test,Mean_pred)
mse_Mean

1816956.6739246522

In [64]:
test_data.head(3)

,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,notRepairedDamage,regionCode,seller,offerType,creatDate,v_0,v_1,v_2,v_3,v_4,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,-0.626175,0.387732,-1.023744,-0.441564,-0.656148,-0.52802,-0.090964,0.613866,-0.322371,1.290430,0.0,0.0,-0.208392,-0.917910,-0.832981,-0.273306,0.687411,0.826899,-0.253673,-0.859081,-0.096178,-0.410154,0.907233,0.956383,-0.086143,-0.804529,0.756892,0.775844
1,-1.565901,-0.574058,-0.264209,0.132000,-0.656148,-0.52802,-0.226682,-1.686465,-0.322371,-0.569587,0.0,0.0,-0.190429,-0.200411,-0.874191,-0.288705,0.593496,-0.910816,0.291727,-0.863742,-0.021692,-0.391268,-0.748213,0.791099,-0.428475,-0.407501,-1.052346,-0.185059
2,1.057539,-0.854581,-0.390799,1.279129,-0.656148,-0.52802,0.001130,-1.942058,-0.322371,-0.426142,0.0,0.0,-0.199411,0.402311,0.945365,-0.356651,-1.250892,0.504024,0.295484,0.894477,-0.619014,0.735603,0.321161,-1.054903,-0.132612,0.363538,1.266359,0.990116


In [48]:
test_data=delete(test_data,"SaleID")

In [50]:
test_data=delete(test_data,"name")

In [52]:
test_data=StandardScaler().fit_transform(test_data)

In [57]:
test_data=pd.DataFrame(test_data)
test_data.columns=data1.columns

In [59]:
#预测

In [60]:
test_rfr_pred1=model_rfr.predict(test_data)

In [62]:
test_rfr_pred1

array([1284.67, 1878.22, 8978.5 , ..., 5293.68, 5174.69, 5691.33])

In [72]:
test_xgb_pred2=model_xgb.predict(np.array(test_data))

In [73]:
test_xgb_pred2

array([1485.1403, 1831.8303, 7977.811 , ..., 5936.838 , 4980.952 ,
       5880.97  ], dtype=float32)

In [76]:
Mean_test_pred=Mean_method(test_rfr_pred1,test_xgb_pred2)

In [77]:
Mean_test_pred

0         1384.905129
1         1855.025161
2         8478.155518
3         1249.992471
4         2030.798325
             ...     
49995     6309.089297
49996    18444.365898
49997     5615.258945
49998     5077.821074
49999     5786.150107
Length: 50000, dtype: float64

In [78]:
Mean_test_pred.to_csv(r"C:\Users\xiongyuan\Desktop\predict.csv")